# Longwave Radiative Transfer Example with PyRTE-RRTMGP

This notebook demonstrates how to use the PyRTE-RRTMGP package to solve a longwave radiative transfer problem. PyRTE-RRTMGP is a Python implementation of the Radiative Transfer for Energetics (RTE).

## Overview

PyRTE-RRTMGP provides a flexible and efficient framework for computing radiative fluxes in planetary atmospheres. This example specifically focuses on:

1. Loading gas optics data for longwave radiation
2. Processing atmospheric data from the RFMIP (Radiative Forcing Model Intercomparison Project)
3. Computing gas optics properties
4. Solving the radiative transfer equation to obtain upward and downward fluxes
5. Validating results against reference solutions generated with the original RTE fortran code

The package leverages xarray and dask for efficient data handling and parallel computation, making it suitable for large-scale atmospheric modeling applications.

## Key Components

- **Gas Optics**: Handles spectral properties of atmospheric gases
- **RTE Solver**: Computes radiative fluxes based on atmospheric properties
- **Data Handling**: Uses xarray for labeled, multi-dimensional data structures

This example demonstrates the workflow for longwave radiative transfer calculations, which are essential for understanding Earth's energy budget and climate modeling.

See the [documentation](https://pyrte-rrtmgp.readthedocs.io/en/latest/) for more information.

## Setup and Configuration

First, we import the necessary libraries and modules. PyRTE-RRTMGP relies on:
- **numpy** and **xarray** for data handling
- Various modules from the `pyrte_rrtmgp` package used to load the data

The key components we'll use are:
- `rrtmgp_gas_optics`: Handles spectral properties of gases
- `rte_solver`: Solves the radiative transfer equation

In [6]:
import numpy as np

from pyrte_rrtmgp.data_types import RFMIPExampleFiles, GasOpticsFiles
from pyrte_rrtmgp.utils import load_rrtmgp_file
from pyrte_rrtmgp import rrtmgp_gas_optics
from pyrte_rrtmgp.rte_solver import rte_solve

## Loading Gas Optics Data

We get the default data files from the package that are available in the [rrtmgp-data](https://github.com/earth-system-radiation/rrtmgp-data) repository.

We're using the longwave gas optics file with 256 g-points (`LW_G256`).

The atmosphere is the RFMIP (Radiative Forcing Model Intercomparison Project) dataset. This dataset contains atmospheric profiles with various combinations of temperature, pressure, and gas concentrations.

In [7]:
gas_optics_lw = rrtmgp_gas_optics.load_gas_optics(gas_optics_file=GasOpticsFiles.LW_G256)
atmosphere = load_rrtmgp_file(RFMIPExampleFiles.RFMIP)

## Computing Gas Optics

Next, we define the gas mapping dictionary that specifies the gas names in the atmosphere dataset, the gas optics file names are the keys in the dictionary and are the default names used internally in the package.

With that we compute the gas optics for the atmosphere `absorption` problem type. The computed gas optics are stored in the `atmosphere` Dataset.

In [8]:
gas_mapping = {
    "h2o": "water_vapor",
    "co2": "carbon_dioxide_GM",
    "o3": "ozone",
    "n2o": "nitrous_oxide_GM",
    "co": "carbon_monoxide_GM",
    "ch4": "methane_GM",
    "o2": "oxygen_GM",
    "n2": "nitrogen_GM",
    "ccl4": "carbon_tetrachloride_GM",
    "cfc11": "cfc11_GM",
    "cfc12": "cfc12_GM",
    "cfc22": "hcfc22_GM",
    "hfc143a": "hfc143a_GM",
    "hfc125": "hfc125_GM",
    "hfc23": "hfc23_GM",
    "hfc32": "hfc32_GM",
    "hfc134a": "hfc134a_GM",
    "cf4": "cf4_GM",
    "no2": "no2",
}

gas_optics_lw.compute_gas_optics(
    atmosphere, problem_type="absorption", gas_name_map=gas_mapping
)
atmosphere["tau"]

<xarray.DataArray 'tau' (site: 100, expt: 18, layer: 60, gpt: 256)> Size: 221MB
array([[[[1.69440385e-08, 4.40707991e-08, 7.45095070e-08, ...,
          3.70267051e-02, 8.74716326e-02, 1.62575165e-01],
         [4.97162082e-08, 1.27982110e-07, 2.11527656e-07, ...,
          3.57600418e-02, 7.92802462e-02, 1.43896031e-01],
         [1.17634963e-07, 3.03783833e-07, 4.96797124e-07, ...,
          4.85102681e-02, 1.02881712e-01, 1.83646585e-01],
         ...,
         [3.08956875e+00, 1.22307171e+01, 2.46596566e+01, ...,
          8.23572753e+00, 9.71891982e+00, 1.16378046e+01],
         [2.16000706e+00, 8.56371768e+00, 1.72862493e+01, ...,
          5.72170060e+00, 6.74478144e+00, 8.07781350e+00],
         [1.45901438e+00, 5.79253138e+00, 1.17042096e+01, ...,
          3.85106368e+00, 4.53565360e+00, 5.43275972e+00]],

        [[1.69440385e-08, 4.40707991e-08, 7.45095070e-08, ...,
          1.63401493e-02, 3.86022300e-02, 7.17462762e-02],
         [4.97162082e-08, 1.27982110e-07, 2.11527656e-07, ...,
          1.57810757e-02, 3.49871380e-02, 6.35018049e-02],
         [1.17634963e-07, 3.03783833e-07, 4.96797124e-07, ...,
          2.14075574e-02, 4.54028051e-02, 8.10452612e-02],
...
         [4.66791589e+00, 1.85885645e+01, 3.75559181e+01, ...,
          9.34560757e+00, 1.08483140e+01, 1.29550499e+01],
         [3.23756552e+00, 1.29002104e+01, 2.60695735e+01, ...,
          6.43414429e+00, 7.46417755e+00, 8.91247721e+00],
         [2.23070389e+00, 8.89052867e+00, 1.79646067e+01, ...,
          4.39864204e+00, 5.10038723e+00, 6.08896947e+00]],

        [[1.67942852e-08, 4.37030454e-08, 7.38028085e-08, ...,
          7.44173667e-03, 1.78419696e-02, 3.31107255e-02],
         [5.20835087e-08, 1.34110952e-07, 2.21162273e-07, ...,
          7.20290868e-03, 1.59312882e-02, 2.87347561e-02],
         [1.16014722e-07, 3.00077657e-07, 4.89560731e-07, ...,
          9.79758639e-03, 2.07474894e-02, 3.68514803e-02],
         ...,
         [4.24422294e+00, 1.68792169e+01, 3.40953285e+01, ...,
          8.03939550e+00, 9.30204914e+00, 1.09629709e+01],
         [2.94478133e+00, 1.17181850e+01, 2.36765020e+01, ...,
          5.53782371e+00, 6.40371041e+00, 7.54692773e+00],
         [2.02961821e+00, 8.07861917e+00, 1.63221793e+01, ...,
          3.78922396e+00, 4.37973044e+00, 5.16166260e+00]]]])
Coordinates:
  * site     (site) int64 800B 0 1 2 3 4 5 6 7 8 ... 91 92 93 94 95 96 97 98 99
  * expt     (expt) int64 144B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
  * gpt      (gpt) int64 2kB 0 1 2 3 4 5 6 7 ... 248 249 250 251 252 253 254 255
Dimensions without coordinates: layer

## Solving the Radiative Transfer Equation

With the gas optics properties computed, we can now solve the radiative transfer equation using the `rte_solve`. This will calculate the upward and downward longwave radiative fluxes for each atmospheric profile.

In [9]:
fluxes = rte_solve(atmosphere, add_to_input=False)
fluxes

<xarray.Dataset> Size: 3MB
Dimensions:              (site: 100, expt: 18, level: 61)
Coordinates:
  * site                 (site) int64 800B 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
  * expt                 (expt) int64 144B 0 1 2 3 4 5 6 ... 12 13 14 15 16 17
Dimensions without coordinates: level
Data variables:
    lw_flux_up_jacobian  (site, expt, level) float64 878kB nan nan ... nan nan
    lw_flux_up           (site, expt, level) float64 878kB 290.0 290.0 ... 439.4
    lw_flux_down         (site, expt, level) float64 878kB 0.0 0.2089 ... 354.7

## Validating Results Against Reference Solutions

Finally, we validate our computed fluxes against reference solutions. The reference data comes from the original RTE-RRTMGP implementation and makes sure that the implementation is correct.

We compare both upward (`rlu`) and downward (`rld`) longwave fluxes to ensure our implementation produces accurate results within the specified error tolerance (`ERROR_TOLERANCE = 1e-7`).

In [10]:
rlu = load_rrtmgp_file(RFMIPExampleFiles.REFERENCE_RLU)
rld = load_rrtmgp_file(RFMIPExampleFiles.REFERENCE_RLD)

assert np.isclose(fluxes["lw_flux_up"].transpose("expt", "site", "level"), rlu["rlu"], atol=1e-7).all(), "Longwave flux up mismatch"
assert np.isclose(fluxes["lw_flux_down"].transpose("expt", "site", "level"), rld["rld"], atol=1e-7).all(), "Longwave flux down mismatch"

print("Longwave radiation transfer calculations validated successfully!")

Longwave radiation transfer calculations validated successfully!
